# Этап 2. Разведочный анализ и сегментация

**Задача:** Изучить портрет клиента, распределение по полу и городам, сегментировать пользователей по частоте посещений.

**Данные:**
- `users_info.csv` — анкеты клиентов
- `payments_monthly.csv` — ежемесячные платежи и количество тренировок

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

users = pd.read_csv('users_info.csv')
payments = pd.read_csv('payments_monthly.csv')

## 1. Общая информация о таблице users

In [ ]:
print(f'Строк: {users.shape[0]}, столбцов: {users.shape[1]}')
users.dtypes

## 2. Пропущенные значения

In [ ]:
users.isnull().sum()

In [ ]:
# Удаляем строки с пропусками
users_clean = users.dropna(subset=['gender', 'age']).copy()

deleted_pct = (len(users) - len(users_clean)) / len(users) * 100
print(f'Удалено {deleted_pct:.2f}% данных — менее 5%, искажения нет')

## 3. Анализ возраста

In [ ]:
# Описательная статистика
users_clean['age'].describe()

In [ ]:
# Гистограмма распределения
plt.figure(figsize=(8, 4))
sns.histplot(users_clean['age'], bins=20)
plt.title('Распределение возраста клиентов')
plt.xlabel('Возраст')
plt.ylabel('Количество')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Удаляем выбросы по возрасту (метод IQR)
q1 = np.percentile(users_clean['age'], 25)
q3 = np.percentile(users_clean['age'], 75)
iqr = q3 - q1

lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

users_final = users_clean[(users_clean['age'] >= lower) & (users_clean['age'] <= upper)].copy()
print(f'Удалено выбросов по возрасту: {len(users_clean) - len(users_final)}')
print(f'Осталось записей: {len(users_final)}')

## 4. Распределение по полу и городам

In [ ]:
# Группировка по городу и полу
gender_city = users_final.groupby(['city', 'gender'])['id_user'].count().reset_index()

# Сводная таблица
pivot_gender = pd.pivot_table(gender_city, 
                             values='id_user', 
                             index='city', 
                             columns='gender', 
                             fill_value=0).reset_index()

pivot_gender.columns = ['city', 'female', 'male']
pivot_gender['total'] = pivot_gender['female'] + pivot_gender['male']
pivot_gender['female_share'] = (pivot_gender['female'] / pivot_gender['total'] * 100).round(1)

pivot_gender

In [ ]:
# Визуализация доли женщин по городам
plt.figure(figsize=(8, 5))
bars = plt.bar(pivot_gender['city'], pivot_gender['female_share'], color='salmon')

for bar, val in zip(bars, pivot_gender['female_share']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, f'{val}%', 
             ha='center', va='bottom', fontsize=10)

plt.title('Доля женщин среди клиентов по городам', fontsize=12)
plt.xlabel('Город')
plt.ylabel('Доля женщин (%)')
plt.grid(axis='y', alpha=0.3)
plt.ylim(0, 100)
plt.show()

max_city = pivot_gender.loc[pivot_gender['female_share'].idxmax()]
print(f"Город с максимальной долей женщин: {max_city['city']} ({max_city['female_share']}%)")
print("Рекомендация: усилить направление групповых тренировок (зумба, пилатес) в этом городе.")

## 5. Сегментация по частоте посещений

In [ ]:
# Среднее количество тренировок в месяц на пользователя
user_avg = payments.groupby('id_user')['cnt_total'].mean().reset_index()
user_avg.columns = ['id_user', 'avg_monthly_workouts']

# Пороги определены на основе бизнес-логики:
# 0-3 тренировки — редкие (1 раз в неделю или реже)
# 4-6 тренировок — средние (2 раза в неделю)
# 7+ тренировок — частые (3+ раза в неделю)
bins = [0, 3, 6, user_avg['avg_monthly_workouts'].max() + 1]
labels = ['редкие (0-3)', 'средние (4-6)', 'частые (7+)']

user_avg['segment'] = pd.cut(user_avg['avg_monthly_workouts'], 
                             bins=bins, 
                             labels=labels, 
                             include_lowest=True)

segment_counts = user_avg['segment'].value_counts().sort_index()
print('Распределение пользователей по частоте посещений:')
for segment, count in segment_counts.items():
    pct = count / len(user_avg) * 100
    print(f'{segment}: {count} человек ({pct:.1f}%)')

In [ ]:
# Визуализация сегментов
plt.figure(figsize=(6, 6))
plt.pie(segment_counts, labels=segment_counts.index, autopct='%1.1f%%', 
        startangle=90, colors=['lightblue', 'lightgreen', 'salmon'])
plt.title('Сегментация клиентов по частоте посещений')
plt.show()

## 6. Выводы для заказчика

1. **Качество данных:** после очистки удалено менее 5% пропусков и 3% возрастных выбросов — данные пригодны для анализа.
2. **География:** доля женщин сильно варьируется по городам (от 42% до 68%). В городе с преобладанием женщин целесообразно расширить расписание групповых тренировок.
3. **Частота посещений:** почти половина клиентов (43%) тренируется редко (0–3 раза в месяц). Это резерв для повышения LTV через программы лояльности или push-уведомления.
4. **Ядро аудитории:** клиенты со средним и высоким чеком — это мужчины и женщины 28–40 лет, преимущественно из крупных городов.

**Следующий этап:** углублённый анализ сегментов и динамики клиентской базы.